### Skolregistret Stockholms stadsarkiv
**Status**: started test webcrawl, version 0.2

* [Denna Notebook](https://github.com/perrohdin/Bildhistoria-Masterdata-och-Wikidata/blob/main/Notebook/Skolregistret%20Stockholms%20stadsarkiv.ipynb)
 * [skapad csv fil](https://github.com/perrohdin/Bildhistoria-Masterdata-och-Wikidata/blob/main/Notebook/SkolregistretStockholmsStadsarkiv.csv) / [raw](https://raw.githubusercontent.com/perrohdin/Bildhistoria-Masterdata-och-Wikidata/main/Notebook/SkolregistretStockholmsStadsarkiv.csv)

Databas copyright okänd, har besökt stadsarkivet och ombads skicka fråga med email
* [sok.stadsarkivet.stockholm.se/Databas/skolregistret](https://sok.stadsarkivet.stockholm.se/Databas/skolregistret/Sok) 1354 poster

**Version history**
* 0.2 fixed bug with matched classes


In [1]:
from datetime import datetime
start_time  = datetime.now()
print("Last run: ", start_time)

Last run:  2022-05-13 08:30:14.654834


In [3]:
import urllib3, re, time
import pandas as pd 
from bs4 import BeautifulSoup
from csv import DictReader, DictWriter
from tqdm import tqdm
req = urllib3.PoolManager()
    
urlbase = "https://sok.stadsarkivet.stockholm.se/Databas/skolregistret/Sok?sidindex="
skolKolumner = ["Url","Namn","Alt. namn","Byggår och arkitekt","Alternativt namn","Annexskolor",
           "Adress","Arkiv på Stadsarkivet","Rektorsområde","Arkiv hos övriga","Historik"]
dfTot = pd.DataFrame(columns=skolKolumner)
for i in tqdm(range(0,1300)):
    try:
        url = urlbase + str(i)
        res = req.request('GET', url)
        soup = BeautifulSoup(res.data, 'html.parser')
        classes = ["arende-table-row arende-table-row-alternating", 
                   "arende-table-row arende-table-row-nonalternating"]
        contents = soup.find_all(class_ = classes)
        i = 0 
        for tablerow in contents:
            for rowLink in tablerow.find_all('a', href=True):
                schoolUrl = rowLink['href'].split("?sidindex")[0]
                namnUrl = rowLink.string
                
             
            td_list = tablerow.find_all("td")
            #print(tablerow)
            namn = td_list[0].text.strip()
            altnamn = td_list[1].text.strip()
            adress = td_list[2].text.strip()
            byggår = td_list[3].text.strip()
            #print(namn,altnamn,adress,byggår)
                
            dfTot = dfTot.append({'Url': schoolUrl, 
                                  'Namn': namn, 
                                  'Alt. namn': altnamn,
                                  'Byggår och arkitekt':byggår,
                                  'Adress':adress
                                  }, ignore_index=True)
    except Exception as e: 
        print(e)


100%|██████████| 1300/1300 [03:34<00:00,  6.07it/s]


In [4]:
dfTot.shape

(1354, 11)

In [5]:
dfTot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1354 entries, 0 to 1353
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Url                    1354 non-null   object
 1   Namn                   1354 non-null   object
 2   Alt. namn              1354 non-null   object
 3   Byggår och arkitekt    1354 non-null   object
 4   Alternativt namn       0 non-null      object
 5   Annexskolor            0 non-null      object
 6   Adress                 1354 non-null   object
 7   Arkiv på Stadsarkivet  0 non-null      object
 8   Rektorsområde          0 non-null      object
 9   Arkiv hos övriga       0 non-null      object
 10  Historik               0 non-null      object
dtypes: object(11)
memory usage: 116.5+ KB


In [6]:
dfTot

,Url,Namn,Alt. namn,Byggår och arkitekt,Alternativt namn,Annexskolor,Adress,Arkiv på Stadsarkivet,Rektorsområde,Arkiv hos övriga,Historik
0,Visa/carlgrenska-waldorfgymnasiet/bf4aeb94-88b...,Carlgrenska Waldorfgymnasiet,,Fristående gymnasieskola\r\n\r\nÖppnad ht 2013...,NaN,NaN,Skebokvarnsvägen 370\r\ntidigare Tegeluddsväge...,NaN,NaN,NaN,NaN
1,Visa/larsboda-ro--1960-1991/7bb2e5c7-e0aa-4754...,Larsboda ro 1960-1991,,Huvudskola:\r\nLarsbodaskolan 1960-1991,NaN,NaN,,NaN,NaN,NaN,NaN
2,Visa/elektrum/5240aebb-c129-4615-9474-006c5b4e...,Elektrum,Electrum,Gymnasium\r\n\r\nAnnex till Vasa gymnasium 198...,NaN,NaN,Kistagången 14-18,NaN,NaN,NaN,NaN
3,Visa/slattgardsskolan/51e42e66-35c7-4a49-9355-...,Slättgårdsskolan,,Grundskola åk 1-9 (före ht 2005 1-6)\r\n\r\nEg...,NaN,NaN,Frimurarvägen 11-17,NaN,NaN,NaN,NaN
4,Visa/lovsatra-resursskola/de119201-dadb-4815-a...,Lövsätra resursskola,Skärholmens skoldaghem / Skärholmens resursskola,Specialskola: Grundskola åk 1-6\r\n\r\nÖppnad ...,NaN,NaN,1971-2002 Falkholmsgränd 41 (tidigare Stånghol...,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1349,Visa/kunskapsskolan-tyreso/e2dbbd7c-a06a-47ce-...,Kunskapsskolan Tyresö,,Fristående grundskola åk 6-9.\r\n\r\nSkolan st...,NaN,NaN,Solkraftsvägen 15,NaN,NaN,NaN,NaN
1350,Visa/sodermalms-privata-skola/0f0c2328-a79d-46...,Södermalms privata skola,,Privatskola,NaN,NaN,1938 anges Folkungagatan 55,NaN,NaN,NaN,NaN
1351,Visa/olof-fryxells-flickskola/7c1a21e8-8f27-45...,Olof Fryxells flickskola,Bataljonspredikanten Olof Fryxells flickskola,Anges finnas 1848 på given adress i Staden mel...,NaN,NaN,Järntorget 80,NaN,NaN,NaN,NaN
1352,Visa/martinskolan/fcb2590d-bda7-4c70-9616-fee8...,Martinskolan,Martinskolan Söders waldorfskola 7 / (Martinsk...,"Fristående Waldorfskola åk F-9.\r\nHuvudman, S...",NaN,NaN,Munstycksvägen 18,NaN,NaN,NaN,NaN


In [7]:
dfTot.to_csv("SkolregistretStockholmsStadsarkiv.csv")